# Introduction
In this example we will simulate bit-interleaved coded modulation (BICM) transmission. The transmitted signal is spread over several subcarriers in the frequency domain, termed orthogonal frequency division multiplexing (OFDM). The radio channel is frequency selective fading channel.


In [ ]:
transport_block_size = config['link']['transport_block_size']
nrof_frames = channel_coeff_freq_domain.cols()
info_bits_uncoded = random.randb(transport_block_size * nrof_frames) # bmat[block_size, nrof_samples]

logging.debug('Channel encode the transmit data bits')

#    info_bits_encoded = codec.encode_bits(info_bits_uncoded, codec='Turbo', par=config['codec'])

info_bits_encoded = codec.turbo_encoder.encode(config['codec'], transport_block_size)(info_bits_uncoded)
encoded_block_size = info_bits_encoded.length() / nrof_frames

interleaver_bin = sequence_interleaver_bin(encoded_block_size)
interleaver_bin.randomize_interleaver_sequence()
interleaver_sequence = interleaver_bin.get_interleaver_sequence()

#    interleaver_sequence = interleaver.interleave_bits(info_bits_encoded, block_size=encoded_block_size)

info_bits_interleaved = interleaver_bin.interleave(info_bits_encoded)

logging.debug('Rate matching the encoded bits')
transmit_block_size = config['link']['nrof_used_subcarriers'] * config['simulation']['nrof_subframe_ofdm_symbols'] * config['modem']['modulation_order']
info_bits_rate_matched = codec.rate.rate_match(transmit_block_size, encoded_block_size)(info_bits_interleaved)

logging.debug('Modulate the encoded bits')
info_symbols_modulated = modem.modulator.modulate_bits(config['modem']['modulation_order'])(info_bits_rate_matched)

logging.debug('Obtain the frequency-domain signal')
transmit_signal_freq_domain = multiplexer.ofdm.multiplex_symbols(config['simulation']['nrof_subframe_ofdm_symbols'],
                                                                 config['simulation']['nrof_total_subcarriers'],
                                                                 info_symbols_modulated)

logging.debug('Apply the channel to the transmitted signal')
received_signal_freq_domain = transmit_signal_freq_domain.elem_mult(channel_coeff_freq_domain)

logging.debug('Add receiver noise')
noise_std_dev = np.sqrt(1.0 / pow(10, 0.1 * snr_db)) # Signal and channel power is normalized to 1 
received_signal_freq_domain_noisy = received_signal_freq_domain + noise_std_dev * randn_c(received_signal_freq_domain.rows(),
                                                                                          received_signal_freq_domain.cols())
#     received_signal_freq_domain_noisy, noise_realization = noise.awgn(noise_var, config['noise']['bias'])(received_signal_freq_domain) 

logging.debug('--Process the received signals--')

logging.debug('Remove the effect of channel')
received_signal_freq_domain_compensated = received_signal_freq_domain_noisy.elem_div(channel_coeff_freq_domain) 

logging.debug('Obtain the time-domain symbols')
received_symbols_modulated = multiplexer.ofdm.de_multiplex_symbols(config['simulation']['nrof_subframe_ofdm_symbols'],
                                                                   config['simulation']['nrof_total_subcarriers'],
                                                                   received_signal_freq_domain_compensated)

logging.debug('Demodulate the received symbols according to the modulation order')
received_soft_values = modem.modulator.demodulate_soft_bits(config['modem']['modulation_order'], noise_std_dev * noise_std_dev)(received_symbols_modulated)

logging.debug('De-rate match the received soft values')
received_soft_values_de_rate_matched = codec.rate.de_rate_match(transmit_block_size, encoded_block_size)(received_soft_values)

interleaver_double = sequence_interleaver_double(encoded_block_size)
interleaver_double.set_interleaver_sequence(interleaver_sequence)
received_soft_values_deinterleaved = interleaver_double.deinterleave(received_soft_values_de_rate_matched, 0)

logging.debug('Channel decode the data bits according to the code rate')
received_bits_decoded = codec.turbo_encoder.decode(config['codec'], transport_block_size)(received_soft_values_deinterleaved)

logging.debug('Count errors')
#     bit_errors = statistics.count_bit_errors(transmit_bits_uncoded, received_bits_decoded)
#     bler = statistics.block_error_ratio(transmit_bits_uncoded, received_bits_decoded, transport_block_size)
#     block_success = statistics.block_success(transmit_bits_uncoded, received_bits_decoded, transport_block_size)

#     '''Determine block success and failures and calculate the block error ratio
#     '''
#     blerc = BLERC(transport_block_size)
#     blerc.count(info_bits_uncoded, received_bits_decoded)
#     bler = blerc.get_errorrate()

bler, block_success = error_counter(info_bits_uncoded, received_bits_decoded, transport_block_size)